In [183]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [184]:
import pandas as pd 
import numpy as np
import re
import string
import nltk
import operator

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from operator import itemgetter

nltk.download('stopwords')
nltk.download('wordnet')
stopwords = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [185]:
train = pd.read_csv("/content/drive/MyDrive/ML/Intern/result.csv",encoding='latin-1')

In [186]:
train.shape

(881, 4)

In [187]:
train.head()

,text,created_at,author/followers_count,author/screen_name
0,"RT @BarackObama: ""We've got to make sure that ...",1395348387,165,tracyscanlan3
1,"Not a cult, at all-&gt; RT @BarackObama: ""We n...",1395348386,1162,bakedflounder
2,RT @YDMO: RT if you agree with @BarackObama: M...,1395348381,1026,mizzoudems
3,"RT @BarackObama: President Obama: Today, more ...",1395348380,174,chantelltelle
4,"RT @BarackObama: ""We've got to make sure that ...",1395348378,234,Germontani


In [188]:
train.tail()

,text,created_at,author/followers_count,author/screen_name
876,RT @steveyknight: It worked! https://t.co/S8Zq...,1394714187,94,AshButta
877,It worked! https://t.co/S8Zq4LKTNe MT @David_C...,1394714116,1381,steveyknight
878,RT @jackpalmer88: MT @David_Cameron: Falafel w...,1394708445,674,Falafel_Junkie
879,MT @David_Cameron: Falafel with the Mayor of B...,1394707348,1197,jackpalmer88
880,Also comes with a healthy dose of fucking the ...,1394702757,203,timfromdagworth


In [189]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 881 entries, 0 to 880
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   text                    881 non-null    object
 1   created_at              881 non-null    int64 
 2   author/followers_count  881 non-null    int64 
 3   author/screen_name      881 non-null    object
dtypes: int64(2), object(2)
memory usage: 27.7+ KB


In [190]:
train.isnull().sum()

text                      0
created_at                0
author/followers_count    0
author/screen_name        0
dtype: int64

In [191]:
train.isnull().values.any()

False

In [192]:
!pip install tweet-preprocessor

In [193]:
def remove_pattern(text, pattern_regex):
    r = re.findall(pattern_regex, text)
    for i in r:
        text = re.sub(i, '', text)
    return text 
train['text2'] = np.vectorize(remove_pattern)(train['text'], "@[\w]*: | *RT*")

In [194]:
def remove_URL(text):
  text = re.sub(r"http\S+", "", text)
  return text
train['text2'] = train['text2'].apply(lambda x: remove_URL(x))

In [195]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text
train['text2'] = train['text2'].apply(lambda x: remove_punct(x))

In [196]:
def tokenization(text):
    text = re.split('\W+', text)
    return text
train['text2'] = train['text2'].apply(lambda x: tokenization(x.lower()))

In [197]:
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
train['text2'] = train['text2'].apply(lambda x: remove_stopwords(x))

In [198]:
wn = nltk.WordNetLemmatizer()
def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text
train['text2'] = train['text2'].apply(lambda x: lemmatizer(x))

In [199]:
train = train[['text2','author/followers_count','text','created_at','author/screen_name']]
train.head()

,text2,author/followers_count,text,created_at,author/screen_name
0,"[, weve, got, make, sure, every, woman, opport...",165,"RT @BarackObama: ""We've got to make sure that ...",1395348387,tracyscanlan3
1,"[cult, allgt, nag, love, â, michelleobama, mom...",1162,"Not a cult, at all-&gt; RT @BarackObama: ""We n...",1395348386,bakedflounder
2,"[, agree, mo, legislature, isnt, right, thing,...",1026,RT @YDMO: RT if you agree with @BarackObama: M...,1395348381,mizzoudems
3,"[, president, obama, today, woman, bread, winn...",174,"RT @BarackObama: President Obama: Today, more ...",1395348380,chantelltelle
4,"[, weve, got, make, sure, every, woman, opport...",234,"RT @BarackObama: ""We've got to make sure that ...",1395348378,Germontani


In [200]:
corpus = []
length = len(train['text2'])
for i in range(0,length):
  w = train['text2'][i]
  w=' '.join(w)
  corpus.append(w)

In [201]:
cv = TfidfVectorizer()
X = cv.fit_transform(corpus).toarray()
print(X.shape)

(881, 1890)


In [202]:
threshold = 0.03
import scipy
from scipy.cluster import  hierarchy
Z = hierarchy.linkage(X, "average", metric="cosine")
C = hierarchy.fcluster(Z, threshold, criterion="distance")

In [ ]:
clusters = C.tolist()
i=0
for x in clusters:
  train["created_at"][i] = x
  i=i+1

In [204]:
dict = {}
for i in range(len(train)):
  x = train['created_at'][i]
  if x not in dict:
    dict[x] = {}
    dict[x]['tweet'] = []
  dict[x]['tweet'].append(train['text'][i])

for i in dict:
  dict[i]['popularity'] = len(dict[i]['tweet'])


In [205]:
resultList = []
for cluster in dict.values():
  resultList.append(cluster)
resultList = sorted(resultList, key=itemgetter('popularity'), reverse=True)

In [206]:
list1 = []
list2 = []
for i in resultList:
  list1.append(i['tweet'][0])
  list2.append(i['popularity'])

In [207]:
df1 = pd.DataFrame(np.array(list1).reshape(len(list1),1), columns = list("A"))

In [208]:
df2 = pd.DataFrame(np.array(list2).reshape(len(list2),1), columns=list("B"))

In [209]:
submission = pd.DataFrame({
        "Tweet": df1['A'],
        "Popularity": df2['B']
    })
submission.to_csv('solution.csv')
print("Total number of different tweets are %d" %len(dict))

Total number of different tweets are 403
